# Sales Analysis

### Questions:
#### What was the best month for sales? How much was earned that month?
#### What City had the highest number of sales?
#### What time should we display advertisements to maximize likelihood of customer's buying product?
#### What items are most often sold together?
#### What product sold the most? Why do you think it sold the most?

In [1]:
import pandas as pd
import os

In [2]:
data = pd.DataFrame()
files = [file for file in os.listdir("./Sales_Data")]

for file in files:
    df = pd.read_csv("./Sales_Data/"+file)
    data = pd.concat([data, df])

data.to_csv("combined_data.csv", index=False)
data.head()

,Order ID,Product,Quantity Ordered,Price Each,Order Date,Purchase Address
0,295665,Macbook Pro Laptop,1,1700,12/30/19 00:01,"136 Church St, New York City, NY 10001"
1,295666,LG Washing Machine,1,600.0,12/29/19 07:03,"562 2nd St, New York City, NY 10001"
2,295667,USB-C Charging Cable,1,11.95,12/12/19 18:21,"277 Main St, New York City, NY 10001"
3,295668,27in FHD Monitor,1,149.99,12/22/19 15:13,"410 6th St, San Francisco, CA 94016"
4,295669,USB-C Charging Cable,1,11.95,12/18/19 12:38,"43 Hill St, Atlanta, GA 30301"


### Question 1: What was the best month for sales? How much was earned that month?

**First we will add a month column to our data frame**

In [3]:
data["Month"] = data["Order Date"].str[0:2]
data["Month"] = data["Month"].astype("int32")

ValueError: invalid literal for int() with base 10: 'Or'

**Finding and removing this "Or" from the Order Date column**

In [4]:
temp_df = data[data["Order Date"].str[0:2] == "Or"]
temp_df.head()

,Order ID,Product,Quantity Ordered,Price Each,Order Date,Purchase Address,Month
254,Order ID,Product,Quantity Ordered,Price Each,Order Date,Purchase Address,Or
705,Order ID,Product,Quantity Ordered,Price Each,Order Date,Purchase Address,Or
1101,Order ID,Product,Quantity Ordered,Price Each,Order Date,Purchase Address,Or
2875,Order ID,Product,Quantity Ordered,Price Each,Order Date,Purchase Address,Or
3708,Order ID,Product,Quantity Ordered,Price Each,Order Date,Purchase Address,Or


In [5]:
data = data[data["Order Date"].str[0:2] != "Or"]

**Trying to add month column again**

In [6]:
data["Month"] = data["Order Date"].str[0:2]
data["Month"] = data["Month"].astype("int32")

ValueError: cannot convert float NaN to integer

**Finding and removing this NaN from the Order Date column**

In [7]:
nan_df = data[data.isna().any(axis=1)]
nan_df.head()

,Order ID,Product,Quantity Ordered,Price Each,Order Date,Purchase Address,Month
264,NaN,NaN,NaN,NaN,NaN,NaN,NaN
648,NaN,NaN,NaN,NaN,NaN,NaN,NaN
680,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1385,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1495,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
data.dropna(how="all",inplace=True)

**Trying to add month column again**

In [9]:
data["Month"] = data["Order Date"].str[0:2]
data["Month"] = data["Month"].astype("int32")

In [10]:
data.head()

,Order ID,Product,Quantity Ordered,Price Each,Order Date,Purchase Address,Month
0,295665,Macbook Pro Laptop,1,1700,12/30/19 00:01,"136 Church St, New York City, NY 10001",12
1,295666,LG Washing Machine,1,600.0,12/29/19 07:03,"562 2nd St, New York City, NY 10001",12
2,295667,USB-C Charging Cable,1,11.95,12/12/19 18:21,"277 Main St, New York City, NY 10001",12
3,295668,27in FHD Monitor,1,149.99,12/22/19 15:13,"410 6th St, San Francisco, CA 94016",12
4,295669,USB-C Charging Cable,1,11.95,12/18/19 12:38,"43 Hill St, Atlanta, GA 30301",12


**Now we need a sales column to denote the total value of each order**

In [11]:
data["Sales"] = data["Quantity Ordered"]*data["Price Each"]

TypeError: can't multiply sequence by non-int of type 'str'

**So we need to convert "Quantity Ordered" and "Price Each" to numeric datatypes**

In [12]:
data["Quantity Ordered"] = pd.to_numeric(data["Quantity Ordered"])
data["Price Each"] = pd.to_numeric(data["Price Each"])

In [13]:
data["Sales"] = data["Quantity Ordered"]*data["Price Each"]

In [14]:
data.head()

,Order ID,Product,Quantity Ordered,Price Each,Order Date,Purchase Address,Month,Sales
0,295665,Macbook Pro Laptop,1,1700.00,12/30/19 00:01,"136 Church St, New York City, NY 10001",12,1700.00
1,295666,LG Washing Machine,1,600.00,12/29/19 07:03,"562 2nd St, New York City, NY 10001",12,600.00
2,295667,USB-C Charging Cable,1,11.95,12/12/19 18:21,"277 Main St, New York City, NY 10001",12,11.95
3,295668,27in FHD Monitor,1,149.99,12/22/19 15:13,"410 6th St, San Francisco, CA 94016",12,149.99
4,295669,USB-C Charging Cable,1,11.95,12/18/19 12:38,"43 Hill St, Atlanta, GA 30301",12,11.95


In [15]:
month_group = data.groupby("Month")
df = month_group.sum()
df["Sales"].sort_values(ascending=False)

Month
12    4613443.34
10    3736726.88
4     3390670.24
11    3199603.20
5     3152606.75
3     2807100.38
7     2647775.76
6     2577802.26
8     2244467.88
2     2202022.42
9     2097560.13
1     1822256.73
Name: Sales, dtype: float64

### **This shows us that December was the best month for sales and it generated a total revenue of 4612443.34**

### Question 2: What City had the highest number of sales?

**We need a city column to group the data**

In [16]:
data.head()

,Order ID,Product,Quantity Ordered,Price Each,Order Date,Purchase Address,Month,Sales
0,295665,Macbook Pro Laptop,1,1700.00,12/30/19 00:01,"136 Church St, New York City, NY 10001",12,1700.00
1,295666,LG Washing Machine,1,600.00,12/29/19 07:03,"562 2nd St, New York City, NY 10001",12,600.00
2,295667,USB-C Charging Cable,1,11.95,12/12/19 18:21,"277 Main St, New York City, NY 10001",12,11.95
3,295668,27in FHD Monitor,1,149.99,12/22/19 15:13,"410 6th St, San Francisco, CA 94016",12,149.99
4,295669,USB-C Charging Cable,1,11.95,12/18/19 12:38,"43 Hill St, Atlanta, GA 30301",12,11.95


In [17]:
def get_city(address):
    add_split = address.split(',')
    city = add_split[-2]+", "+add_split[-1].split(' ')[1]
    return city

data["City"] = data["Purchase Address"].apply(get_city)
data.head()

,Order ID,Product,Quantity Ordered,Price Each,Order Date,Purchase Address,Month,Sales,City
0,295665,Macbook Pro Laptop,1,1700.00,12/30/19 00:01,"136 Church St, New York City, NY 10001",12,1700.00,"New York City, NY"
1,295666,LG Washing Machine,1,600.00,12/29/19 07:03,"562 2nd St, New York City, NY 10001",12,600.00,"New York City, NY"
2,295667,USB-C Charging Cable,1,11.95,12/12/19 18:21,"277 Main St, New York City, NY 10001",12,11.95,"New York City, NY"
3,295668,27in FHD Monitor,1,149.99,12/22/19 15:13,"410 6th St, San Francisco, CA 94016",12,149.99,"San Francisco, CA"
4,295669,USB-C Charging Cable,1,11.95,12/18/19 12:38,"43 Hill St, Atlanta, GA 30301",12,11.95,"Atlanta, GA"


In [18]:
city_group = data.groupby("City")
df = city_group.count()
df["Sales"].sort_values(ascending=False)

City
 San Francisco, CA    44732
 Los Angeles, CA      29605
 New York City, NY    24876
 Boston, MA           19934
 Atlanta, GA          14881
 Dallas, TX           14820
 Seattle, WA          14732
 Portland, OR         10010
 Austin, TX            9905
 Portland, ME          2455
Name: Sales, dtype: int64

### **This shows us that San Francisco had the highest number of sales at 44732 orders.**

**We can also check out the revenue generated by each city to see how they performed**

In [19]:
df = city_group.sum()
df["Sales"].sort_values(ascending=False)

City
 San Francisco, CA    8262203.91
 Los Angeles, CA      5452570.80
 New York City, NY    4664317.43
 Boston, MA           3661642.01
 Atlanta, GA          2795498.58
 Dallas, TX           2767975.40
 Seattle, WA          2747755.48
 Portland, OR         1870732.34
 Austin, TX           1819581.75
 Portland, ME          449758.27
Name: Sales, dtype: float64

### **This shows us that the median cost of items bought is relatively similar for each city since the order of cities didn't change from no. of orders to total revenue generated**

### Question 3: What time should we display advertisements to maximize likelihood of customer's buying product?

In [20]:
data.head()

,Order ID,Product,Quantity Ordered,Price Each,Order Date,Purchase Address,Month,Sales,City
0,295665,Macbook Pro Laptop,1,1700.00,12/30/19 00:01,"136 Church St, New York City, NY 10001",12,1700.00,"New York City, NY"
1,295666,LG Washing Machine,1,600.00,12/29/19 07:03,"562 2nd St, New York City, NY 10001",12,600.00,"New York City, NY"
2,295667,USB-C Charging Cable,1,11.95,12/12/19 18:21,"277 Main St, New York City, NY 10001",12,11.95,"New York City, NY"
3,295668,27in FHD Monitor,1,149.99,12/22/19 15:13,"410 6th St, San Francisco, CA 94016",12,149.99,"San Francisco, CA"
4,295669,USB-C Charging Cable,1,11.95,12/18/19 12:38,"43 Hill St, Atlanta, GA 30301",12,11.95,"Atlanta, GA"


**First we need to convert the string order date into a date time format for easy accessibility of dates and times**

In [21]:
data["Order Date"] = pd.to_datetime(data["Order Date"])

In [22]:
data.head()

,Order ID,Product,Quantity Ordered,Price Each,Order Date,Purchase Address,Month,Sales,City
0,295665,Macbook Pro Laptop,1,1700.00,2019-12-30 00:01:00,"136 Church St, New York City, NY 10001",12,1700.00,"New York City, NY"
1,295666,LG Washing Machine,1,600.00,2019-12-29 07:03:00,"562 2nd St, New York City, NY 10001",12,600.00,"New York City, NY"
2,295667,USB-C Charging Cable,1,11.95,2019-12-12 18:21:00,"277 Main St, New York City, NY 10001",12,11.95,"New York City, NY"
3,295668,27in FHD Monitor,1,149.99,2019-12-22 15:13:00,"410 6th St, San Francisco, CA 94016",12,149.99,"San Francisco, CA"
4,295669,USB-C Charging Cable,1,11.95,2019-12-18 12:38:00,"43 Hill St, Atlanta, GA 30301",12,11.95,"Atlanta, GA"


**Now we would like to have a separate hours column to group data by**

In [23]:
data["Order Hour"] = data["Order Date"].dt.hour

In [24]:
data.head()

,Order ID,Product,Quantity Ordered,Price Each,Order Date,Purchase Address,Month,Sales,City,Order Hour
0,295665,Macbook Pro Laptop,1,1700.00,2019-12-30 00:01:00,"136 Church St, New York City, NY 10001",12,1700.00,"New York City, NY",0
1,295666,LG Washing Machine,1,600.00,2019-12-29 07:03:00,"562 2nd St, New York City, NY 10001",12,600.00,"New York City, NY",7
2,295667,USB-C Charging Cable,1,11.95,2019-12-12 18:21:00,"277 Main St, New York City, NY 10001",12,11.95,"New York City, NY",18
3,295668,27in FHD Monitor,1,149.99,2019-12-22 15:13:00,"410 6th St, San Francisco, CA 94016",12,149.99,"San Francisco, CA",15
4,295669,USB-C Charging Cable,1,11.95,2019-12-18 12:38:00,"43 Hill St, Atlanta, GA 30301",12,11.95,"Atlanta, GA",12


In [25]:
order_hour_group = data.groupby("Order Hour")
df = order_hour_group.count()
df["Sales"].sort_values(ascending=False)

Order Hour
19    12905
12    12587
11    12411
18    12280
20    12228
13    12129
14    10984
10    10944
21    10921
17    10899
16    10384
15    10175
22     8822
9      8748
23     6275
8      6256
7      4011
0      3910
6      2482
1      2350
5      1321
2      1243
4       854
3       831
Name: Sales, dtype: int64

### This shows us that the peak sale hours are around 10am to 2pm and 6pm to 8pm. So an advertisement in or just before these time slots should be the most profitable

### Question 4: What items are most often sold together?

In [26]:
data.head()

,Order ID,Product,Quantity Ordered,Price Each,Order Date,Purchase Address,Month,Sales,City,Order Hour
0,295665,Macbook Pro Laptop,1,1700.00,2019-12-30 00:01:00,"136 Church St, New York City, NY 10001",12,1700.00,"New York City, NY",0
1,295666,LG Washing Machine,1,600.00,2019-12-29 07:03:00,"562 2nd St, New York City, NY 10001",12,600.00,"New York City, NY",7
2,295667,USB-C Charging Cable,1,11.95,2019-12-12 18:21:00,"277 Main St, New York City, NY 10001",12,11.95,"New York City, NY",18
3,295668,27in FHD Monitor,1,149.99,2019-12-22 15:13:00,"410 6th St, San Francisco, CA 94016",12,149.99,"San Francisco, CA",15
4,295669,USB-C Charging Cable,1,11.95,2019-12-18 12:38:00,"43 Hill St, Atlanta, GA 30301",12,11.95,"Atlanta, GA",12


**Products inside of a single order have the same Order ID**

In [27]:
duplicated_df = data[data["Order ID"].duplicated(keep=False)]
duplicated_df

,Order ID,Product,Quantity Ordered,Price Each,Order Date,Purchase Address,Month,Sales,City,Order Hour
16,295681,Google Phone,1,600.00,2019-12-25 12:37:00,"79 Elm St, Boston, MA 02215",12,600.00,"Boston, MA",12
17,295681,USB-C Charging Cable,1,11.95,2019-12-25 12:37:00,"79 Elm St, Boston, MA 02215",12,11.95,"Boston, MA",12
18,295681,Bose SoundSport Headphones,1,99.99,2019-12-25 12:37:00,"79 Elm St, Boston, MA 02215",12,99.99,"Boston, MA",12
19,295681,Wired Headphones,1,11.99,2019-12-25 12:37:00,"79 Elm St, Boston, MA 02215",12,11.99,"Boston, MA",12
36,295698,Vareebadd Phone,1,400.00,2019-12-13 14:32:00,"175 1st St, New York City, NY 10001",12,400.00,"New York City, NY",14
...,...,...,...,...,...,...,...,...,...,...
13592,222882,AA Batteries (4-pack),1,3.84,2019-06-18 11:37:00,"990 Walnut St, Los Angeles, CA 90001",6,3.84,"Los Angeles, CA",11
13594,222884,Google Phone,1,600.00,2019-06-04 12:06:00,"819 Pine St, New York City, NY 10001",6,600.00,"New York City, NY",12
13595,222884,USB-C Charging Cable,1,11.95,2019-06-04 12:06:00,"819 Pine St, New York City, NY 10001",6,11.95,"New York City, NY",12
13614,222903,iPhone,1,700.00,2019-06-07 22:21:00,"685 Hickory St, New York City, NY 10001",6,700.00,"New York City, NY",22


In [28]:
order_id_group = duplicated_df.groupby("Order ID")
result = order_id_group.agg(', '.join).reset_index()
result

/var/folders/58/hxsmfxz10w50ybr1bjmly6c00000gn/T/ipykernel_2026/971420714.py:2: FutureWarning: ['Quantity Ordered', 'Price Each', 'Order Date', 'Month', 'Sales', 'Order Hour'] did not aggregate successfully. If any error is raised this will raise in a future version of pandas. Drop these columns/ops to avoid this warning.
  result = order_id_group.agg(', '.join).reset_index()


,Order ID,Product,Purchase Address,City
0,141275,"USB-C Charging Cable, Wired Headphones","610 Walnut St, Austin, TX 73301, 610 Walnut St...","Austin, TX, Austin, TX"
1,141290,"Apple Airpods Headphones, AA Batteries (4-pack)","4 1st St, Los Angeles, CA 90001, 4 1st St, Los...","Los Angeles, CA, Los Angeles, CA"
2,141365,"Vareebadd Phone, Wired Headphones","20 Dogwood St, New York City, NY 10001, 20 Dog...","New York City, NY, New York City, NY"
3,141384,"Google Phone, USB-C Charging Cable","223 Jackson St, Boston, MA 02215, 223 Jackson ...","Boston, MA, Boston, MA"
4,141450,"Google Phone, Bose SoundSport Headphones","521 Park St, San Francisco, CA 94016, 521 Park...","San Francisco, CA, San Francisco, CA"
...,...,...,...,...
7131,319536,"Macbook Pro Laptop, Wired Headphones","346 Elm St, Seattle, WA 98101, 346 Elm St, Sea...","Seattle, WA, Seattle, WA"
7132,319556,"Google Phone, Wired Headphones","848 7th St, Austin, TX 73301, 848 7th St, Aust...","Austin, TX, Austin, TX"
7133,319584,"iPhone, Wired Headphones","537 Sunset St, Portland, OR 97035, 537 Sunset ...","Portland, OR, Portland, OR"
7134,319596,"iPhone, Lightning Charging Cable","436 14th St, New York City, NY 10001, 436 14th...","New York City, NY, New York City, NY"


In [29]:
duplicated_df = duplicated_df[["Order ID", "Product"]]
duplicated_df

,Order ID,Product
16,295681,Google Phone
17,295681,USB-C Charging Cable
18,295681,Bose SoundSport Headphones
19,295681,Wired Headphones
36,295698,Vareebadd Phone
...,...,...
13592,222882,AA Batteries (4-pack)
13594,222884,Google Phone
13595,222884,USB-C Charging Cable
13614,222903,iPhone


In [30]:
order_id_group = duplicated_df.groupby("Order ID")
result = order_id_group.agg(', '.join).reset_index()
result

,Order ID,Product
0,141275,"USB-C Charging Cable, Wired Headphones"
1,141290,"Apple Airpods Headphones, AA Batteries (4-pack)"
2,141365,"Vareebadd Phone, Wired Headphones"
3,141384,"Google Phone, USB-C Charging Cable"
4,141450,"Google Phone, Bose SoundSport Headphones"
...,...,...
7131,319536,"Macbook Pro Laptop, Wired Headphones"
7132,319556,"Google Phone, Wired Headphones"
7133,319584,"iPhone, Wired Headphones"
7134,319596,"iPhone, Lightning Charging Cable"


In [31]:
product_group = result.groupby("Product")
df = product_group.count()
df["Order ID"].sort_values(ascending=False)

Product
iPhone, Lightning Charging Cable                        882
Google Phone, USB-C Charging Cable                      856
iPhone, Wired Headphones                                361
Vareebadd Phone, USB-C Charging Cable                   312
Google Phone, Wired Headphones                          303
                                                       ... 
LG Dryer, 27in FHD Monitor                                1
LG Dryer, 27in 4K Gaming Monitor                          1
Google Phone, Wired Headphones, USB-C Charging Cable      1
Google Phone, Wired Headphones, Macbook Pro Laptop        1
iPhone, iPhone                                            1
Name: Order ID, Length: 366, dtype: int64

### This shows us that phones and the cables/plug-in accessories are sold together most often.

### Question 5: What product sold the most? Why do you think it sold the most?

In [32]:
data.head()

,Order ID,Product,Quantity Ordered,Price Each,Order Date,Purchase Address,Month,Sales,City,Order Hour
0,295665,Macbook Pro Laptop,1,1700.00,2019-12-30 00:01:00,"136 Church St, New York City, NY 10001",12,1700.00,"New York City, NY",0
1,295666,LG Washing Machine,1,600.00,2019-12-29 07:03:00,"562 2nd St, New York City, NY 10001",12,600.00,"New York City, NY",7
2,295667,USB-C Charging Cable,1,11.95,2019-12-12 18:21:00,"277 Main St, New York City, NY 10001",12,11.95,"New York City, NY",18
3,295668,27in FHD Monitor,1,149.99,2019-12-22 15:13:00,"410 6th St, San Francisco, CA 94016",12,149.99,"San Francisco, CA",15
4,295669,USB-C Charging Cable,1,11.95,2019-12-18 12:38:00,"43 Hill St, Atlanta, GA 30301",12,11.95,"Atlanta, GA",12


In [33]:
product_group = data.groupby("Product")
df = product_group.sum()
df["Quantity Ordered"].sort_values(ascending=False)

Product
AAA Batteries (4-pack)        31017
AA Batteries (4-pack)         27635
USB-C Charging Cable          23975
Lightning Charging Cable      23217
Wired Headphones              20557
Apple Airpods Headphones      15661
Bose SoundSport Headphones    13457
27in FHD Monitor               7550
iPhone                         6849
27in 4K Gaming Monitor         6244
34in Ultrawide Monitor         6199
Google Phone                   5532
Flatscreen TV                  4819
Macbook Pro Laptop             4728
ThinkPad Laptop                4130
20in Monitor                   4129
Vareebadd Phone                2068
LG Washing Machine              666
LG Dryer                        646
Name: Quantity Ordered, dtype: int64

In [34]:
df = product_group.mean()
df["Price Each"].sort_values(ascending=True)

Product
AAA Batteries (4-pack)           2.99
AA Batteries (4-pack)            3.84
USB-C Charging Cable            11.95
Wired Headphones                11.99
Lightning Charging Cable        14.95
Bose SoundSport Headphones      99.99
20in Monitor                   109.99
27in FHD Monitor               149.99
Apple Airpods Headphones       150.00
Flatscreen TV                  300.00
34in Ultrawide Monitor         379.99
27in 4K Gaming Monitor         389.99
Vareebadd Phone                400.00
Google Phone                   600.00
LG Washing Machine             600.00
LG Dryer                       600.00
iPhone                         700.00
ThinkPad Laptop                999.99
Macbook Pro Laptop            1700.00
Name: Price Each, dtype: float64

### This shows us that batteries, cables and headphones are the most sold products. This is because they are cheaper, everyday use items and are prone to run out or get damaged easily due to extensive use. On the other hand products like washers, driers, monitors, laptops and phones are sold less often because they are expensive, last a long amount of time and are not as heavily used as cables and batteries